In [1]:
#importing pandas library
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1       CONFIRMED              0              0              0              0   
2  FALSE POSITIVE              0              1              0              0   
3  FALSE POSITIVE              0              1              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0    9.488036     2.775000e-05    -2.775000e-05   170.538750   
1   54.418383     2.479000e-04    -2.479000e-04   162.513840   
2   19.899140     1.494000e-05    -1.494000e-05   175.850252   
3    1.736952     2.630000e-07    -2.630000e-07   170.307565   
4    2.525592     3.761000e-06    -3.761000e-06   171.595550   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.002160  ...           -81.0      4.467           0.064   
1          0.003520  ...           -81.0      4.467           0.064   
2          0.000581  ...          -176.0      4.544           0.044   
3          0.000115  ...          -174.0      4.564           0.053   
4          0.001130  ...          -211.0      4.438           0.070   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.096     0.927          0.105         -0.061  291.93423   
2          -0.176     0.868          0.233         -0.078  297.00482   
3          -0.168     0.791          0.201         -0.067  285.53461   
4          -0.210     1.046          0.334         -0.133  288.75488   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.141651      15.347  
2  48.134129      15.436  
3  48.285210      15.597  
4  48.226200      15.509  

[5 rows x 41 columns]

In [4]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"].head()

0         CONFIRMED
1         CONFIRMED
2    False_Positive
3    False_Positive
4         CONFIRMED
Name: koi_disposition, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [7]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


# Create and train the Deep Learning Model using Keras

In [10]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X_train_scaled, y_train_categorical, epochs=50, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 1s - loss: 0.5161 - acc: 0.7461
Epoch 2/50
 - 1s - loss: 0.3628 - acc: 0.8150
Epoch 3/50
 - 1s - loss: 0.3494 - acc: 0.8224
Epoch 4/50
 - 1s - loss: 0.3396 - acc: 0.8297
Epoch 5/50
 - 1s - loss: 0.3314 - acc: 0.8393
Epoch 6/50
 - 1s - loss: 0.3250 - acc: 0.8481
Epoch 7/50
 - 1s - loss: 0.3191 - acc: 0.8522
Epoch 8/50
 - 1s - loss: 0.3155 - acc: 0.8532
Epoch 9/50
 - 1s - loss: 0.3053 - acc: 0.8638
Epoch 10/50
 - 1s - loss: 0.3015 - acc: 0.8654
Epoch 11/50
 - 1s - loss: 0.2988 - acc: 0.8675
Epoch 12/50
 - 1s - loss: 0.2951 - acc: 0.8684
Epoch 13/50
 - 1s - loss: 0.2959 - acc: 0.8681
Epoch 14/50
 - 1s - loss: 0.2904 - acc: 0.8734
Epoch 15/50
 - 1s - loss: 0.2836 - acc: 0.8759
Epoch 16/50
 - 1s - loss: 0.2836 - acc: 0.8745
Epoch 17/50
 - 1s - loss: 0.2864 - acc: 0.8715
Epoch 18/50
 - 1s - loss: 0.2773 - acc: 0.8811
Epoch 19/50
 - 1s - loss: 0.2745 - acc: 0.8785
Epoch 20/50
 - 1s - loss: 0.2716 - acc: 0.8830
Epoch 21/50
 - 1s - l

In [14]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.3007972549650501, Accuracy: 0.8732845378598268


In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [16]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CONFIRMED']
Actual Labels: ['False_Positive', 'CANDIDATE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE']
